# Clean Data

Merge dataframes, subset columns, create new columns

## Set Up

Import packages/libraries and data

In [ ]:
import sys
import os

sys.path.append(os.path.abspath("../"))
from src.config import BASE_PATH
from src.data_utils import import_raw_data_dict, get_feature_lists

from src.clean_filter import merge_dfs, clean_dfs
import pandas as pd
import numpy as np
from shutil import rmtree
import warnings

## Merge DFs

Must clean up column names and ensure all desired variables are present

In [ ]:
data_path = BASE_PATH / "data" / "raw" / "cpt_filtered"
data_dict = import_raw_data_dict(data_path)
combined_df, combined_df_w_codes = merge_dfs(data_dict)

## Clean resulting categorical instances

In [ ]:
replace_dict = {
    "SEX": {
        "nan": None,
        "non-bi": None,
        "NULL": None,
        "non-binary": None,
        "None": None,
        "male": 0,
        "female": 1,
    },
    "RACE_NEW": {
        "Unknown/Not Reported": "otherUnknown",
        "Unknown": "otherUnknown",
        "nan": "otherUnknown",
        "Native Hawaiian or Other Pacific Islander": "Native Hawaiian or Pacific Islander",
        "Asian,Some Other Race": "otherUnknown",
        "Native Hawaiian or Pacifi": "Native Hawaiian or Pacific Islander",
        "American Indian or Alaska": "American Indian or Alaska Native",
        "White,Black or African American": "otherUnknown",
        "White,Asian": "otherUnknown",
        "White,Some Other Race": "otherUnknown",
        "Race combinations with low frequency": "otherUnknown",
        "Native Hawaiian or Other Pacific Islander,Asian": "otherUnknown",
        "White,American Indian or Alaska Native": "otherUnknown",
        "White,Native Hawaiian or Other Pacific Islander": "otherUnknown",
        "Black or African American,American Indian or Alaska Native": "otherUnknown",
        "White,Native Hawaiian or Other Pacific Islander,As": "otherUnknown",
        "White,Native Hawaiian or Other Pacific Islander,Asian": "otherUnknown",
        "Black or African American,American Indian or Alask": "otherUnknown",
        "Black or African American,Asian,Some Other Race": "otherUnknown",
        "White,Unknown/Not Reported": "otherUnknown",
        "Race combinations with low frequency": "otherUnknown",
        "NULL": "otherUnknown",
        "Some Other Race": "otherUnknown",
        "Middle Eastern or North African": "otherUnknown",
        "White,Hispanic or Latino": "otherUnknown",
        "White,Middle Eastern or North African": "otherUnknown",
        "Hispanic or Latino": "otherUnknown",
        "Black or African American,Hispanic or Latino": "otherUnknown",
        "Hispanic or Latino,Some Other Race": "otherUnknown",
        "Black or African American,Middle Eastern or North African": "otherUnknown",
    },
    "ETHNICITY_HISPANIC": {
        "N": "noUnknown",
        "U": "noUnknown",
        "Y": "Yes",
        "No": "noUnknown",
        "Unknown": "noUnknown",
        "nan": "noUnknown",
        "Unk": "noUnknown",
        "NULL": "noUnknown",
        "None": "noUnknown",
    },
    # "AGE": { #will do this individually
    #     "90+": "90",
    # },
    "ANESTHES": {
        "Monitored Anesthesia Care": "MAC",
        "MAC/IV Sedation": "MAC",
        "Local": "otherUnknown",
        "Regional": "otherUnknown",
        "Other": "otherUnknown",
        "Spinal": "otherUnknown",
        "Epidural": "otherUnknown",
        "None": "otherUnknown",
        "Unknown": "otherUnknown",
        "nan": "otherUnknown",
        "NULL": "otherUnknown",
        None: "otherUnknown",
    },
    "SURGSPEC": {
        "Gynecology": "otherUnknown",
        "Vascular": "otherUnknown",
        "Otolaryngology (ENT)": "otherUnknown",
        "Neurosurgery": "otherUnknown",
        "Orthopedics": "otherUnknown",
        "Thoracic": "otherUnknown",
        "Urology": "otherUnknown",
        "Cardiac Surgery": "otherUnknown",
        "Obstetrics": "otherUnknown",
        "Interventional Radiologist": "otherUnknown",
        "Other": "otherUnknown",
    },
    "HEIGHT": {-99.0: None},
    "WEIGHT": {-99.0: None},
    "DIABETES": {"NON-INSULIN": "ORAL"},
    "SMOKE": {"nan": None},
    "ASCITES": {"Ye": "Yes"},
    "HXCHF": {"Ye": "Yes"},
    "HYPERMED": {"nan": None},
    "DIALYSIS": {"Ye": "Yes"},
    "DISCANCR": {"nan": None},
    "URGENCY": {
        "No": "Elective",
        "Emergent": "Urgent",
        "Yes": "Urgent",
        "NUL": None,
        "nan": None,
    },
    "ASACLAS": {  # Will impute these NAs later
        "None assigned": None,
        "nan": None,
        "None": None,
        "1-No Disturb": 1,
        "2-Mild Disturb": 2,
        "3-Severe Disturb": 3,
        "4-Life Threat": 4,
        "5-Moribund": 4,
    },
    "YRDEATH": {-99: "No", "-99.0": "No", "-99": "No", None: "No"},
    "SUPINFEC": {"No Complication": "No", "Superficial Incisional SSI": "Yes"},
    "WNDINFD": {
        "No Complication": "No",
        "Deep Incisional SSI": "Yes",
        None: "No",
    },
    "WNDINF": {
        None: "No",  # if 21-24 change to unknown
    },
    "ORGSPCSSI": {"No Complication": "No", "Organ/Space SSI": "Yes"},
    "DEHIS": {
        "No Complication": "No",
        "Wound Disruption": "Yes",
        "0": "No",
        "1": "Yes",
        "2": "Yes",
    },
    "OUPNEUMO": {"No Complication": "No", "Pneumonia": "Yes"},
    "REINTUB": {
        "No Complication": "No",
        "Unplanned Intubation": "Yes",
        "Unplanned Intub": "Yes",
    },
    "PULEMBOL": {"No Complication": "No", "Pulmonary Embolism": "Yes"},
    "FAILWEAN": {"No Complication": "No", "On Ventilator greater than 48 Hours": "Yes"},
    "URNINFEC": {"No Complication": "No", "Urinary Tract Infection": "Yes"},
    "CNSCVA": {"No Complication": "No", "Stroke/CVA": "Yes"},
    "CDARREST": {
        "No Complication": "No",
        "Cardiac Arrest Requiring CPR": "Yes",
        "Cardiac Arrest": "Yes",
    },
    "CDMI": {
        "No Complication": "No",
        "Myocardial Infarction": "Yes",
        "Myocardial Infa": "Yes",
    },
    "OTHBLEED": {
        "No Complication": "No",
        "Transfusions/Intraop/Postop": "Yes",
        "Blood Transfusion": "Yes",
        "Bleeding/Transfusions": "Yes",
    },
    "OTHDVT": {
        "No Complication": "No",
        "DVT Requiring Therapy": "Yes",
        "Venous Thrombosis Requiring Therapy": "Yes",
        "DVT Requiring Therap": "Yes",
    },
    "OTHSYSEP": {"No Complication": "No", "Sepsis": "Yes"},
    "OTHSESHOCK": {"No Complication": "No", "Septic Shock": "Yes"},
    "READ": {
        "nan": "No",
        np.nan: "No",
    },  # --> will change to unknown IF 2008-2010 later
    "UNPLREAD": {
        np.nan: "No",
        None: "No",
        "nan": "No",
        "None": "No",
    },  # --> will change to unknown IF 2008-2011 later
    "UNPLREOP": {
        None: "No"
    },  # only 2011 had NULL values + no "No" entries --> assume NULL = No here
    "DYSPNEA": {
        "MODERATE EXERTION": "Yes",
        "AT REST": "Yes",
        "nan": "No",
        None: "No",  # will change to unknown later if 21-24
    },
    "FNSTATUS2": {
        "Unknown": "otherUnknown",
        "Partially Dependent": "Dependent",
        "Totally Dependent": "Dependent",
        "Partially D": "Dependent",
        "Totally Dep": "Dependent",
    },
    "RENAFAIL": {
        "Ye": "Yes",
        None: "No",  # will change to unknown later if 21
    },
    "PRSEPIS": {
        "None": "No",
        "nan": "No",
        "SIRS": "Yes",
        "Sepsis": "Yes",
        "Septic Shock": "Yes",
        "Septic": "Yes",
        "NULL": "No",
        None: "No",
    },
    "RENAINSF": {
        "No Complication": "No",
        "Progressive Renal Insufficiency": "Yes",
        "Postop  Renal Insufficiency": "Yes",
        "Postop† Renal†Insufficiency": "Yes",
        "Progressive Ren": "Yes",
        "Postop\xa0 Renal\xa0Insufficiency": "Yes",
        None: "No",  # Will change to unknown later if 21
    },
    "OPRENAFL": {
        "No Complication": "No",
        "Acute Renal Failure": "Yes",
        "Postop Dialysis": "Yes",
        "Acute Renal Fai": "Yes",
    },
    "DISCHDEST": {
        "Home": "HomePermRes",
        "Home/Permanent Residence": "HomePermRes",
        "Unknown": "otherUnknown",
        "Facility Which was Home": "HomePermRes",
        "Separate Acute Care": "Acute Care",
        "Acute Care Hospital": "Acute Care",
        "Unskilled Facility Not Home": "otherUnknown",
        "Hospice": "otherUnknown",
        "Other Facility": "otherUnknown",
        "Against Medical Advice (AMA)": "otherUnknown",
        "None": "otherUnknown",
        "NULL": "otherUnknown",
        "nan": "otherUnknown",
        "Unskilled Facility Not": "otherUnknown",
        "Multi-level Senior Community": "otherUnknown",
        None: "otherUnknown",
    },
    "WTLOSS": {None: "No"},  # Will change later to unknown if 21-24,
    "UNPLREOP": {
        "nan": "No"
    },  # only null for 2011 (and has no "No" entries), assume means NO
}

na_drop_cols = [
    "SEX",
    "SMOKE",
    "HYPERMED",
    "DISCANCR",
    "URGENCY",
    "UNPLREOP",  # if mortality == 1, make "No", otherwise drop
]

recon_cols = [
    "immediateCPT",
    "delayedCPT",
    "teinsertionCPT",
    "teexpanderCPT",
    "freeCPT",
    "latCPT",
    "SinTramCPT",
    "SinTramSuperCPT",
    "BiTramCPT",
    "MastoCPT",
    "BreastRedCPT",
    "FatGraftCPT",
    "AdjTisTransCPT",
    "AugProsImpCPT",
    "OtherReconTechCPT",
    "RevRecBreastCPT",
]

mast_cols = ["partialCPT", "subsimpleCPT", "radicalCPT", "modifiedRadicalCPT"]

surg_ind_cols = [
    "CarcinomaICD",
    "MalignantICD",
    "MetastaticICD",
    "ProphylacticICD",
    "AbBreastICD",
    "BenignICD",
    "InflOtherICD",
    "CongICD",
    "AbsICD",
]

drop_cols = [
    "PODIAG",
    "PODIAGTX",
    "PODIAG10",
    "PODIAGTX10",
    "PRNCPTX",
    "CPT",
    "OTHERCPT1",
    "OTHERCPT2",
    "OTHERCPT3",
    "OTHERCPT4",
    "OTHERCPT5",
    "OTHERCPT6",
    "OTHERCPT7",
    "OTHERCPT8",
    "OTHERCPT9",
    "OTHERCPT10",
    "CONCPT1",
    "CONCPT2",
    "CONCPT3",
    "CONCPT4",
    "CONCPT5",
    "CONCPT6",
    "CONCPT7",
    "CONCPT8",
    "CONCPT9",
    "CONCPT10",
]

num_cols = [
    "AGE",
    "OPERYR",
    "HEIGHT",
    "WEIGHT",
    "PRALBUM",
    "PRWBC",
    "PRHCT",
    "PRPLATE",
    "OPTIME",
    "TOTHLOS",
]

df_replaced = clean_dfs(
    combined_df=combined_df_w_codes,
    replace_dict=replace_dict,
    na_drop_cols=na_drop_cols,
    recon_cols=recon_cols,
    mast_cols=mast_cols,
    surg_ind_cols=surg_ind_cols,
    num_cols=num_cols,
    drop_cols=drop_cols,
)

In [ ]:
feat_lists = get_feature_lists(df_replaced)
num_cols = feat_lists["numerical_cols"]
ord_cols = feat_lists["ordinal_cols"]
nom_cols = feat_lists["nominal_cols"]
bin_cols = feat_lists["binary_cols"]
### Ensure no more NAs in non-numerical cols ###
assert df_replaced[nom_cols + bin_cols].isna().sum().sum() == 0

## Create Target Variables

In [ ]:
surg_outcome_cols = ["SUPINFEC", "WNDINFD", "ORGSPCSSI", "DEHIS", "OTHBLEED"]

df_replaced["AnySurgComp"] = (  # type: ignore
    (df_replaced[surg_outcome_cols] == 1).any(axis=1).astype(int)  # type: ignore
)
# Medical outcomes
med_outcome_cols = [
    "OUPNEUMO",
    "REINTUB",
    "FAILWEAN",
    "URNINFEC",
    "CNSCVA",
    "CDARREST",
    "CDMI",
    "OTHSYSEP",
    "OTHSESHOCK",
    "RENAINSF",
    "OPRENAFL",
]

df_replaced["AnyMedComp"] = (  # type: ignore
    (df_replaced[med_outcome_cols] == 1).any(axis=1).astype(int)  # type: ignore
)
vte_cols = ["OTHDVT", "PULEMBOL"]
df_replaced["VTE"] = (  # type: ignore
    (df_replaced[vte_cols] == 1).any(axis=1).astype(int)  # type: ignore
)
## Get Target Variables
outcome_list = ["MORTALITY", "UNPLREOP", "VTE", "AnySurgComp", "AnyMedComp"]
outcome_dict = {}
for outcome in outcome_list:
    outcome_dict[outcome] = df_replaced[outcome]

## Export

Outcomes

In [ ]:
outcome_df = pd.DataFrame(outcome_dict)
outcome_path = BASE_PATH / "data" / "processed" / "outcome_df.parquet"
if outcome_path.exists():
    outcome_path.unlink()
    warnings.warn(f"Over-writing outcome data at {outcome_path}")
outcome_path.parent.mkdir(exist_ok=True, parents=True)
outcome_df.to_parquet(outcome_path, index=True)

X dfs

In [ ]:
x_path = BASE_PATH / "data" / "raw" / "cleaned" / "NSQIP_mast_combined.parquet"
if x_path.parent.exists():
    rmtree(x_path.parent)
    warnings.warn(f"Over-writing x data at {x_path}")
x_path.parent.mkdir(exist_ok=True, parents=True)

df_replaced.to_parquet(x_path)